## Training Script

### Dataloader

In [15]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms

class ClutteredMNISTDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Custom dataset for Cluttered MNIST.
        :param root_dir: Root directory of the dataset (e.g., "dataset/cluttered_mnist")
        :param transform: Optional torchvision transforms to apply to the images
        """
        self.root_dir = root_dir
        self.transform = transform

        # Gather all image paths and their labels
        self.data = []
        for label in range(10):  # Assuming labels are 0-9
            label_dir = os.path.join(root_dir, str(label))
            if os.path.isdir(label_dir):
                for file_name in os.listdir(label_dir):
                    if file_name.endswith(('.png', '.jpg', '.jpeg')):
                        file_path = os.path.join(label_dir, file_name)
                        self.data.append((file_path, label))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        """
        Retrieve an image and its label at the specified index.
        :param idx: Index of the data point
        :return: Tuple (image, label)
        """
        image_path, label = self.data[idx]
        image = Image.open(image_path).convert('L')  # Convert to grayscale

        if self.transform:
            image = self.transform(image)

        return image, label

# Define transforms for the dataset
transform = transforms.Compose([
    transforms.Resize((100, 100)),  # Resize images to 100x100
    transforms.ToTensor(),          # Convert images to PyTorch tensors
    transforms.Normalize((0.1307,), (0.3081,))  # Normalize using MNIST mean and std
])

# Create dataset
dataset_dir = "dataset/cluttered_mnist"
cluttered_mnist_dataset = ClutteredMNISTDataset(root_dir=dataset_dir, transform=transform)

# Split dataset into train and test (80% train, 20% test)
train_size = int(0.9 * len(cluttered_mnist_dataset))
test_size = len(cluttered_mnist_dataset) - train_size
train_dataset, test_dataset = random_split(cluttered_mnist_dataset, [train_size, test_size])

# DataLoader for batching and shuffling
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Example: Iterate through the DataLoader
for images, labels in train_loader:
    print("Training batch of images shape:", images.shape)  # (batch_size, 1, 100, 100)
    print("Training batch of labels shape:", labels.shape)  # (batch_size,)
    break

for images, labels in test_loader:
    print("Test batch of images shape:", images.shape)  # (batch_size, 1, 100, 100)
    print("Test batch of labels shape:", labels.shape)  # (batch_size,)
    break

print(f"train_size: {train_size}, test_size: {test_size}")
64*900

Training batch of images shape: torch.Size([64, 1, 100, 100])
Training batch of labels shape: torch.Size([64])
Test batch of images shape: torch.Size([64, 1, 100, 100])
Test batch of labels shape: torch.Size([64])
train_size: 54000, test_size: 6000


57600

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from glimpse import GlimpseModel

"""print('the code god was here')"""
# Define the RNN model
class MNISTRNN(nn.Module):
    def __init__(self, crop_size, hidden_size, num_layers, num_classes):
        super(MNISTRNN, self).__init__()
        self.crop_size = crop_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # RNN to process the crops
        self.rnn = nn.RNN(crop_size**2, hidden_size, num_layers, batch_first=True)
        self.fc_class = nn.Linear(hidden_size, num_classes)  # Class prediction
        self.fc_action = nn.Linear(hidden_size, 2)  # Next crop center prediction

    def crop(self, padded_image, center):
        """
        Crop a region around the given center from the padded image.
        :param padded_image: Padded input image (batch_size, 1, padded_size, padded_size)
        :param center: Crop centers (batch_size, 2)
        :return: Cropped image regions (batch_size, crop_size, crop_size)
        """
        crop_size = self.crop_size
        half_crop = crop_size // 2

        # Compute cropping indices
        x_start = (center[:, 0] - half_crop).long()
        x_end = (center[:, 0] + half_crop).long()
        y_start = (center[:, 1] - half_crop).long()
        y_end = (center[:, 1] + half_crop).long()

        # Perform efficient tensor slicing
        crops = torch.stack([
            padded_image[b, :, y_start[b].item():y_end[b].item(), x_start[b].item():x_end[b].item()]
            for b in range(padded_image.size(0))
        ])
        return crops

    def forward(self, image, center, h0):
        """
        Forward pass with dynamic cropping and RNN processing.
        :param image: Full input image (batch_size, 1, 28, 28)
        :param center: Initial crop centers (batch_size, 2)
        :param h0: Initial hidden state (num_layers, batch_size, hidden_size)
        :return: Class prediction, next center, hidden state
        """
        # batch_size = image.size(0)
        # crop_size = self.crop_size

        # Pad the image to avoid boundary issues
        # padding = crop_size // 2
        # padded_image = nn.functional.pad(image, (padding, padding, padding, padding), mode='constant', value=0)

        # Extract crops
        # crops = self.crop(padded_image, center)  # (batch_size, 1, crop_size, crop_size)
        # crops = crops.view(batch_size, -1)  # Flatten the crop (batch_size, crop_size^2)

        # # Process with RNN
        # crops = crops.unsqueeze(1)  # Add sequence dimension (batch_size, seq_len=1, crop_size^2)
        input = image.view(-1, 1, 28*28)
        
        out, hn = self.rnn(input, h0)

        # Predict class and next crop center
        class_pred = self.fc_class(out[:, -1, :])  # Class prediction
        action_pred = self.fc_action(out[:, -1, :])  # Action (next crop center)

        return class_pred, action_pred, hn

In [2]:
# Hyperparameters
crop_size = 20
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 64
learning_rate = 0.001
num_epochs = 10
num_steps = 3  # RNN steps per image

# Load MNIST dataset
train_dataset = datasets.MNIST(root='./dataset/raw_mnist', train=True, download=True, transform=transforms.ToTensor())
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='./dataset/raw_mnist', train=False, download=True, transform=transforms.ToTensor())
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [3]:
# Create model, optimizer, and loss functions
model = MNISTRNN(28, hidden_size, num_layers, num_classes)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion_class = nn.CrossEntropyLoss()
criterion_action = nn.MSELoss()  # For predicting the next center

## Train That Bad Boy

In [4]:
# Training loop
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        batch_size = images.size(0)

        # Initialize the hidden state and center
        h0 = torch.zeros(num_layers, batch_size, hidden_size).to(images.device)
        centers = torch.tensor([[14, 14]] * batch_size).float().to(images.device)  # Initial center

        loss = 0
        for step in range(num_steps):
            # Forward pass
            class_pred, action_pred, h0 = model(images, centers, h0)

            # Compute losses
            loss_class = criterion_class(class_pred, labels)
            # loss_action = criterion_action(action_pred, centers)  # Target is to stay at the initial center
            loss += loss_class

            # Update centers for the next step
            # centers = torch.clip(action_pred, min=crop_size // 2, max=28 + crop_size // 2)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

Epoch [1/10], Step [100/938], Loss: 0.5515
Epoch [1/10], Step [200/938], Loss: 0.7980
Epoch [1/10], Step [300/938], Loss: 0.6671
Epoch [1/10], Step [400/938], Loss: 0.7717
Epoch [1/10], Step [500/938], Loss: 1.2448
Epoch [1/10], Step [600/938], Loss: 0.9178
Epoch [1/10], Step [700/938], Loss: 0.3193
Epoch [1/10], Step [800/938], Loss: 1.0799
Epoch [1/10], Step [900/938], Loss: 0.1862
Epoch [2/10], Step [100/938], Loss: 0.4516
Epoch [2/10], Step [200/938], Loss: 0.1952
Epoch [2/10], Step [300/938], Loss: 0.4004
Epoch [2/10], Step [400/938], Loss: 0.2467
Epoch [2/10], Step [500/938], Loss: 0.3296
Epoch [2/10], Step [600/938], Loss: 0.3747
Epoch [2/10], Step [700/938], Loss: 0.2626
Epoch [2/10], Step [800/938], Loss: 0.2526
Epoch [2/10], Step [900/938], Loss: 0.0651
Epoch [3/10], Step [100/938], Loss: 0.4308
Epoch [3/10], Step [200/938], Loss: 0.3599
Epoch [3/10], Step [300/938], Loss: 0.4499
Epoch [3/10], Step [400/938], Loss: 0.1712
Epoch [3/10], Step [500/938], Loss: 0.2011
Epoch [3/10

In [6]:
# Evaluate the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        batch_size = images.size(0)
        h0 = torch.zeros(num_layers, batch_size, hidden_size).to(images.device)
        
        for step in range(num_steps):
            class_pred, action_pred, h0 = model(images, None, h0)

        _, predicted = torch.max(class_pred.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')

Accuracy of the model on the test images: 97.25%
